In [1]:
import re,math
import numpy as np
import pandas as pd
from helpers import get_inputs

In [2]:
# DAY 1

nums = { 
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

def calib_value(pattern,s):
    g = lambda x: int(nums.get(x,x))
    x = list(map(re.findall(pattern,s).__getitem__,[0,-1]))
    return g(x[0])*10+g(x[1])
    
day1 = get_inputs(1)

#part1
pattern1 = '\d'
d1p1 = sum([calib_value(pattern1,s) for s in day1])

#part2
pattern2 = '(?=(\d|one|two|three|four|five|six|seven|eight|nine))'
d1p2 = sum([calib_value(pattern2,s) for s in day1])

print(f'p1: {d1p1}\np2: {d1p2}')

p1: 54953
p2: 53868


In [4]:
# DAY 2

from functools import reduce

t2 = '''Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green'''.split('\n')

class CubeGame:

    def __init__(self,data):
        self.data = data
        self.seq = self.draw_seq()
        self.id = self.get_id()

    def get_id(self):
        return int(re.findall('Game (\d+)',self.data)[0])

    def draw_seq(self):
        seq = []
        for color in ['red','green','blue']:
            seq.append([int(x) or 0 for x in re.findall(f'(\d+) {color}',self.data)])
        return seq
    
    def is_possible(self,pool):
        # pool: tuple[int] for (red, green, blue)
        for idx,d in enumerate(self.seq):
            if pool[idx] < (max(d) if d else 0):
                return 0
        return 1
    
    @property
    def min_pool(self):
        return [max(x or [0]) for x in self.seq]

pool = (12,13,14)

d2p1 = 0
d2p2 = 0

for game in get_inputs(2):
    g = CubeGame(game)
    d2p1+= g.is_possible(pool)*g.id
    d2p2+= reduce(lambda x,y: x*y , g.min_pool)

print(f'p1: {d2p1}\np2: {d2p2}') 

p1: 2528
p2: 67363


In [116]:
# DAY 3
t3 = '''467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''.split('\n')

class Engine:
    def __init__(self,data):
        self.data = data
        self.numbers = self.get_address(pattern='(\d+)')
        self.stars = self.get_address(pattern='(\*)')
        self.part_numbers = []
        self._get_part_numbers()

    def get_address(self,pattern):
        '''
        Returns tuple of (row#,start_idx,end_idx)
        '''
        address = []
        for idx,row in enumerate(self.data):
            x = re.finditer(pattern,row)
            for i in x:
                address.append(tuple([idx,*i.span()]))
        return address 
    
    def _get_part_numbers(self):
        for x in self.numbers:
            neighbors = ''
            n = int(self.data[x[0]][x[1]:x[2]])
            neighbors += self.data[max(0,x[0]-1)][max(0,x[1]-1):x[2]+1]
            neighbors += self.data[max(0,x[0])][max(0,x[1]-1):x[2]+1]
            neighbors += self.data[max(0,min(x[0]+1,len(self.data[0])-1))][max(0,max(0,x[1]-1)):min(x[2]+1,len(self.data[0])-1)]
            if  re.findall('[^0-9.]',neighbors):
                self.part_numbers.append(n)

    def gear_ratios(self):
        r = []
        for s in self.stars:
            adj_parts = [int(self.data[x[0]][x[1]:x[2]]) for x in self.numbers if abs(x[0]-s[0])<=1 and x[1]-1<=s[1]<=x[2]]
            if len(adj_parts)>1:
                r.append (reduce(lambda x,y: x*y, adj_parts))
        return r
    
e = Engine(get_inputs(3))
print(f'p1: {sum(e.part_numbers)}\np2: {sum(e.gear_ratios())}') 

p1: 551094
p2: 80179647
